In [1]:
#cleaned-up TF-IDF approach

#keyword processing

In [2]:
pip install -U textblob

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install langdetect

In [4]:
import pandas as pd
import numpy as np

# keyword extraction and clean up
import nltk
from rake_nltk import Rake
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
import re

#Text Blob for sentiment analysis
from textblob import TextBlob

#tf-idf model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics.pairwise import linear_kernel

import string
import random
from PIL import Image
import requests
from io import BytesIO

from langdetect import detect

In [5]:
url= "https://ralston-hotel-data-bucket.s3.amazonaws.com/Hotel_Reviews_Clean.csv"

In [6]:
hotel_df = pd.read_csv(url)

In [7]:
hotel_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Netherlands,Negative
1,1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Netherlands,Positive
2,2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968,Netherlands,Positive
3,3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968,Netherlands,Negative
4,4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-24,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968,Netherlands,Positive


In [8]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512470 entries, 0 to 512469
Data columns (total 20 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Unnamed: 0                                  512470 non-null  int64  
 1   Hotel_Address                               512470 non-null  object 
 2   Additional_Number_of_Scoring                512470 non-null  int64  
 3   Review_Date                                 512470 non-null  object 
 4   Average_Score                               512470 non-null  float64
 5   Hotel_Name                                  512470 non-null  object 
 6   Reviewer_Nationality                        512470 non-null  object 
 7   Negative_Review                             512470 non-null  object 
 8   Review_Total_Negative_Word_Counts           512470 non-null  int64  
 9   Total_Number_of_Reviews                     512470 non-null  int64  
 

In [9]:
# combine reviews into one field
hotel_df["All_Reviews"] = hotel_df['Positive_Review'].astype(str) +"-"+ hotel_df["Negative_Review"]
hotel_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment,All_Reviews
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Netherlands,Negative,Only the park outside of the hotel was beauti...
1,1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Ireland,No Negative,0,1403,...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Netherlands,Positive,No real complaints the hotel was great great ...
2,2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968,Netherlands,Positive,Location was good and staff were ok It is cut...
3,3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968,Netherlands,Negative,Great location in nice surroundings the bar a...
4,4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-24,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,...,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968,Netherlands,Positive,Amazing location and building Romantic settin...


In [10]:
#create random sample of original dataset - 5% sample
hotel_sample_df = hotel_df.sample(frac=.05)

In [11]:
hotel_sample_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment,All_Reviews
300147,300675,Coopers Row City of London London EC3N 2BQ UK,267,2016-01-30,8.7,Grange City Hotel,United Kingdom,Mostly everything rude staff useless manageme...,36,872,...,3,2,3.8,"[' Leisure trip ', ' Couple ', ' Double Room '...",551 day,51.511011,-0.077125,UK,Negative,Location - Mostly everything rude staff usele...
150245,150337,33 37 Hogarth Road Kensington and Chelsea Lond...,989,2016-11-13,8.4,Park Grand London Kensington,United Kingdom,Nothing,2,4660,...,19,4,9.2,"[' Leisure trip ', ' Group ', ' Superior Doubl...",263 day,51.493847,-0.191758,UK,Positive,The staff were really lovely and looked after...
321973,322501,Excel 2 Festoon Way Royal Victoria Dock Newham...,853,2016-08-10,8.4,DoubleTree By Hilton London Excel,Switzerland,The way they welcome us did not get breakfast,10,2726,...,4,7,3.3,"[' Couple ', ' Queen Guest Room ', ' Stayed 2 ...",358 day,51.507377,0.038657,UK,Negative,Staffs not welcoming- The way they welcome us...
157275,157367,34 rue de Moscou 8th arr 75008 Paris France,141,2016-07-03,8.9,Best Western Premier Kapital Op ra,United States of America,Bathroom bedroom proportions somewhat off bat...,33,1444,...,9,9,9.6,"[' Leisure trip ', ' Couple ', ' Standard Doub...",396 day,48.881442,2.323559,France,Positive,The staff were incredible Very helpful with e...
227152,227534,64 76 Hendon Way London NW2 2NL UK,471,2016-12-05,7.9,Best Western Palm Hotel,United Kingdom,Pillows could have been softer,7,2037,...,8,1,9.2,"[' Leisure trip ', ' Couple ', ' Executive Dou...",241 day,51.563476,-0.204276,UK,Positive,Lovely modern bathroom and good toiletries - ...


In [12]:
hotel_sample_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25624 entries, 300147 to 481338
Data columns (total 21 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Unnamed: 0                                  25624 non-null  int64  
 1   Hotel_Address                               25624 non-null  object 
 2   Additional_Number_of_Scoring                25624 non-null  int64  
 3   Review_Date                                 25624 non-null  object 
 4   Average_Score                               25624 non-null  float64
 5   Hotel_Name                                  25624 non-null  object 
 6   Reviewer_Nationality                        25624 non-null  object 
 7   Negative_Review                             25624 non-null  object 
 8   Review_Total_Negative_Word_Counts           25624 non-null  int64  
 9   Total_Number_of_Reviews                     25624 non-null  int64  
 10  Posi

In [13]:
hotel_sample_df.reset_index()

,index,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,...,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment,All_Reviews
0,300147,300675,Coopers Row City of London London EC3N 2BQ UK,267,2016-01-30,8.7,Grange City Hotel,United Kingdom,Mostly everything rude staff useless manageme...,36,...,3,2,3.8,"[' Leisure trip ', ' Couple ', ' Double Room '...",551 day,51.511011,-0.077125,UK,Negative,Location - Mostly everything rude staff usele...
1,150245,150337,33 37 Hogarth Road Kensington and Chelsea Lond...,989,2016-11-13,8.4,Park Grand London Kensington,United Kingdom,Nothing,2,...,19,4,9.2,"[' Leisure trip ', ' Group ', ' Superior Doubl...",263 day,51.493847,-0.191758,UK,Positive,The staff were really lovely and looked after...
2,321973,322501,Excel 2 Festoon Way Royal Victoria Dock Newham...,853,2016-08-10,8.4,DoubleTree By Hilton London Excel,Switzerland,The way they welcome us did not get breakfast,10,...,4,7,3.3,"[' Couple ', ' Queen Guest Room ', ' Stayed 2 ...",358 day,51.507377,0.038657,UK,Negative,Staffs not welcoming- The way they welcome us...
3,157275,157367,34 rue de Moscou 8th arr 75008 Paris France,141,2016-07-03,8.9,Best Western Premier Kapital Op ra,United States of America,Bathroom bedroom proportions somewhat off bat...,33,...,9,9,9.6,"[' Leisure trip ', ' Couple ', ' Standard Doub...",396 day,48.881442,2.323559,France,Positive,The staff were incredible Very helpful with e...
4,227152,227534,64 76 Hendon Way London NW2 2NL UK,471,2016-12-05,7.9,Best Western Palm Hotel,United Kingdom,Pillows could have been softer,7,...,8,1,9.2,"[' Leisure trip ', ' Couple ', ' Executive Dou...",241 day,51.563476,-0.204276,UK,Positive,Lovely modern bathroom and good toiletries - ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25619,440056,441974,Scarsdale Place Kensington Kensington and Chel...,1831,2016-04-05,8.1,Copthorne Tara Hotel London Kensington,United Kingdom,Queuing for breakfast and tea served from a j...,30,...,11,1,6.7,"[' Leisure trip ', ' Couple ', ' Superior Twin...",485 day,51.499046,-0.191707,UK,Positive,The location was very good Room was very clea...
25620,200276,200658,5 Olympic Way Wembley Brent London HA9 0NP UK,832,2016-09-04,8.9,Novotel London Wembley,Ireland,No Negative,0,...,16,20,9.2,"[' Solo traveler ', ' Superior Queen Room ', '...",333 day,51.561127,-0.278726,UK,Positive,Staff were wonderful great location and beaut...
25621,53096,53096,146 Praed Street Westminster Borough London W2...,687,2016-01-12,8.2,Hilton London Paddington,United Kingdom,No Negative,0,...,5,1,10.0,"[' Leisure trip ', ' Family with young childre...",569 day,51.515632,-0.175737,UK,Positive,Excellent Reception Staff -No Negative
25622,256317,256699,86 Rue Saint Denis 1st arr 75001 Paris France,18,2017-06-29,9.0,Snob Hotel by Elegancia,Spain,nowhere to store your toiletries in bathroom ...,16,...,19,18,9.2,"[' Business trip ', ' Solo traveler ', ' Class...",35 days,48.862272,2.349571,France,Positive,good size for the price and it being paris sm...


In [14]:
#data pre-processing to remove stop words, punctuation, white space, and convert all words to lower case from combined reviews
hotel_sample_pp_df = hotel_sample_df.copy()
key_words = []
name_key_words = []

r = Rake()

for index, row in hotel_sample_pp_df.iterrows():
    r.extract_keywords_from_text(row['All_Reviews'])
    key_words_dict_scores = r.get_word_degrees()
    keywords = list(key_words_dict_scores.keys())
    
    key_words.append(keywords)

    r.extract_keywords_from_text(row['Hotel_Name'])
    key_words_dict_scores = r.get_word_degrees()
    name_keywords = list(key_words_dict_scores.keys())
    
    name_key_words.append(name_keywords)
    
hotel_sample_pp_df["key_words"] = key_words
hotel_sample_pp_df["name_key_words"] = name_key_words

hotel_sample_pp_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Reviewer_Score,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment,All_Reviews,key_words,name_key_words
300147,300675,Coopers Row City of London London EC3N 2BQ UK,267,2016-01-30,8.7,Grange City Hotel,United Kingdom,Mostly everything rude staff useless manageme...,36,872,...,3.8,"[' Leisure trip ', ' Couple ', ' Double Room '...",551 day,51.511011,-0.077125,UK,Negative,Location - Mostly everything rude staff usele...,"[location, mostly, everything, rude, staff, us...","[grange, city, hotel]"
150245,150337,33 37 Hogarth Road Kensington and Chelsea Lond...,989,2016-11-13,8.4,Park Grand London Kensington,United Kingdom,Nothing,2,4660,...,9.2,"[' Leisure trip ', ' Group ', ' Superior Doubl...",263 day,51.493847,-0.191758,UK,Positive,The staff were really lovely and looked after...,"[staff, really, lovely, looked, us, recognised...","[park, grand, london, kensington]"
321973,322501,Excel 2 Festoon Way Royal Victoria Dock Newham...,853,2016-08-10,8.4,DoubleTree By Hilton London Excel,Switzerland,The way they welcome us did not get breakfast,10,2726,...,3.3,"[' Couple ', ' Queen Guest Room ', ' Stayed 2 ...",358 day,51.507377,0.038657,UK,Negative,Staffs not welcoming- The way they welcome us...,"[staffs, welcoming, way, welcome, us, get, bre...","[doubletree, hilton, london, excel]"
157275,157367,34 rue de Moscou 8th arr 75008 Paris France,141,2016-07-03,8.9,Best Western Premier Kapital Op ra,United States of America,Bathroom bedroom proportions somewhat off bat...,33,1444,...,9.6,"[' Leisure trip ', ' Couple ', ' Standard Doub...",396 day,48.881442,2.323559,France,Positive,The staff were incredible Very helpful with e...,"[staff, incredible, helpful, everything, bathr...","[best, western, premier, kapital, op, ra]"
227152,227534,64 76 Hendon Way London NW2 2NL UK,471,2016-12-05,7.9,Best Western Palm Hotel,United Kingdom,Pillows could have been softer,7,2037,...,9.2,"[' Leisure trip ', ' Couple ', ' Executive Dou...",241 day,51.563476,-0.204276,UK,Positive,Lovely modern bathroom and good toiletries - ...,"[lovely, modern, bathroom, good, toiletries, p...","[best, western, palm, hotel]"


In [15]:
hotel_sample_pp_clean_df = hotel_sample_pp_df.copy()

In [16]:
# clean up keywords: drop symbols and numbers, as well as find root words

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)


hotel_sample_pp_clean_df['clean_keywords']=hotel_sample_pp_clean_df['key_words'].map(lambda s:preprocess(s)) 
hotel_sample_pp_clean_df.head()


,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment,All_Reviews,key_words,name_key_words,clean_keywords
300147,300675,Coopers Row City of London London EC3N 2BQ UK,267,2016-01-30,8.7,Grange City Hotel,United Kingdom,Mostly everything rude staff useless manageme...,36,872,...,"[' Leisure trip ', ' Couple ', ' Double Room '...",551 day,51.511011,-0.077125,UK,Negative,Location - Mostly everything rude staff usele...,"[location, mostly, everything, rude, staff, us...","[grange, city, hotel]",location mostly everything rude staff useless ...
150245,150337,33 37 Hogarth Road Kensington and Chelsea Lond...,989,2016-11-13,8.4,Park Grand London Kensington,United Kingdom,Nothing,2,4660,...,"[' Leisure trip ', ' Group ', ' Superior Doubl...",263 day,51.493847,-0.191758,UK,Positive,The staff were really lovely and looked after...,"[staff, really, lovely, looked, us, recognised...","[park, grand, london, kensington]",staff really lovely looked recognised friend b...
321973,322501,Excel 2 Festoon Way Royal Victoria Dock Newham...,853,2016-08-10,8.4,DoubleTree By Hilton London Excel,Switzerland,The way they welcome us did not get breakfast,10,2726,...,"[' Couple ', ' Queen Guest Room ', ' Stayed 2 ...",358 day,51.507377,0.038657,UK,Negative,Staffs not welcoming- The way they welcome us...,"[staffs, welcoming, way, welcome, us, get, bre...","[doubletree, hilton, london, excel]",staffs welcoming way welcome get breakfast
157275,157367,34 rue de Moscou 8th arr 75008 Paris France,141,2016-07-03,8.9,Best Western Premier Kapital Op ra,United States of America,Bathroom bedroom proportions somewhat off bat...,33,1444,...,"[' Leisure trip ', ' Couple ', ' Standard Doub...",396 day,48.881442,2.323559,France,Positive,The staff were incredible Very helpful with e...,"[staff, incredible, helpful, everything, bathr...","[best, western, premier, kapital, op, ra]",staff incredible helpful everything bathroom b...
227152,227534,64 76 Hendon Way London NW2 2NL UK,471,2016-12-05,7.9,Best Western Palm Hotel,United Kingdom,Pillows could have been softer,7,2037,...,"[' Leisure trip ', ' Couple ', ' Executive Dou...",241 day,51.563476,-0.204276,UK,Positive,Lovely modern bathroom and good toiletries - ...,"[lovely, modern, bathroom, good, toiletries, p...","[best, western, palm, hotel]",lovely modern bathroom good toiletries pillows...


In [17]:
#hotel_sample_pp_clean_df = hotel_sample_pp_clean_df.drop(columns=['key_words'])
#hotel_sample_pp_clean_df.head()

#sentiment analysis

In [18]:
hotel_sub_df = hotel_sample_pp_clean_df.copy()

In [19]:
# run text blob in a loop

polarities = []
subjectivities = []

for index, row in hotel_sub_df.iterrows():
    sentence = row['All_Reviews']
    analysis = TextBlob(sentence).sentiment
    polarity = analysis[0]
    subjectivity = analysis[1]

    # populate lists
    polarities.append(polarity)
    subjectivities.append(subjectivity)


hotel_sub_df['sentiment_polarity'] = polarities
hotel_sub_df['sentiment_subjectivity']= subjectivities

hotel_sub_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,lat,lng,Hotel_Country,Sentiment,All_Reviews,key_words,name_key_words,clean_keywords,sentiment_polarity,sentiment_subjectivity
300147,300675,Coopers Row City of London London EC3N 2BQ UK,267,2016-01-30,8.7,Grange City Hotel,United Kingdom,Mostly everything rude staff useless manageme...,36,872,...,51.511011,-0.077125,UK,Negative,Location - Mostly everything rude staff usele...,"[location, mostly, everything, rude, staff, us...","[grange, city, hotel]",location mostly everything rude staff useless ...,-0.251667,0.536667
150245,150337,33 37 Hogarth Road Kensington and Chelsea Lond...,989,2016-11-13,8.4,Park Grand London Kensington,United Kingdom,Nothing,2,4660,...,51.493847,-0.191758,UK,Positive,The staff were really lovely and looked after...,"[staff, really, lovely, looked, us, recognised...","[park, grand, london, kensington]",staff really lovely looked recognised friend b...,0.566667,0.766667
321973,322501,Excel 2 Festoon Way Royal Victoria Dock Newham...,853,2016-08-10,8.4,DoubleTree By Hilton London Excel,Switzerland,The way they welcome us did not get breakfast,10,2726,...,51.507377,0.038657,UK,Negative,Staffs not welcoming- The way they welcome us...,"[staffs, welcoming, way, welcome, us, get, bre...","[doubletree, hilton, london, excel]",staffs welcoming way welcome get breakfast,0.800000,0.900000
157275,157367,34 rue de Moscou 8th arr 75008 Paris France,141,2016-07-03,8.9,Best Western Premier Kapital Op ra,United States of America,Bathroom bedroom proportions somewhat off bat...,33,1444,...,48.881442,2.323559,France,Positive,The staff were incredible Very helpful with e...,"[staff, incredible, helpful, everything, bathr...","[best, western, premier, kapital, op, ra]",staff incredible helpful everything bathroom b...,0.153016,0.318413
227152,227534,64 76 Hendon Way London NW2 2NL UK,471,2016-12-05,7.9,Best Western Palm Hotel,United Kingdom,Pillows could have been softer,7,2037,...,51.563476,-0.204276,UK,Positive,Lovely modern bathroom and good toiletries - ...,"[lovely, modern, bathroom, good, toiletries, p...","[best, western, palm, hotel]",lovely modern bathroom good toiletries pillows...,0.466667,0.550000


In [25]:
hotel_sub_df.reset_index()

,index,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,...,lat,lng,Hotel_Country,Sentiment,All_Reviews,key_words,name_key_words,clean_keywords,sentiment_polarity,sentiment_subjectivity
0,300147,300675,Coopers Row City of London London EC3N 2BQ UK,267,2016-01-30,8.7,Grange City Hotel,United Kingdom,Mostly everything rude staff useless manageme...,36,...,51.511011,-0.077125,UK,Negative,Location - Mostly everything rude staff usele...,"[location, mostly, everything, rude, staff, us...","[grange, city, hotel]",location mostly everything rude staff useless ...,-0.251667,0.536667
1,150245,150337,33 37 Hogarth Road Kensington and Chelsea Lond...,989,2016-11-13,8.4,Park Grand London Kensington,United Kingdom,Nothing,2,...,51.493847,-0.191758,UK,Positive,The staff were really lovely and looked after...,"[staff, really, lovely, looked, us, recognised...","[park, grand, london, kensington]",staff really lovely looked recognised friend b...,0.566667,0.766667
2,321973,322501,Excel 2 Festoon Way Royal Victoria Dock Newham...,853,2016-08-10,8.4,DoubleTree By Hilton London Excel,Switzerland,The way they welcome us did not get breakfast,10,...,51.507377,0.038657,UK,Negative,Staffs not welcoming- The way they welcome us...,"[staffs, welcoming, way, welcome, us, get, bre...","[doubletree, hilton, london, excel]",staffs welcoming way welcome get breakfast,0.800000,0.900000
3,157275,157367,34 rue de Moscou 8th arr 75008 Paris France,141,2016-07-03,8.9,Best Western Premier Kapital Op ra,United States of America,Bathroom bedroom proportions somewhat off bat...,33,...,48.881442,2.323559,France,Positive,The staff were incredible Very helpful with e...,"[staff, incredible, helpful, everything, bathr...","[best, western, premier, kapital, op, ra]",staff incredible helpful everything bathroom b...,0.153016,0.318413
4,227152,227534,64 76 Hendon Way London NW2 2NL UK,471,2016-12-05,7.9,Best Western Palm Hotel,United Kingdom,Pillows could have been softer,7,...,51.563476,-0.204276,UK,Positive,Lovely modern bathroom and good toiletries - ...,"[lovely, modern, bathroom, good, toiletries, p...","[best, western, palm, hotel]",lovely modern bathroom good toiletries pillows...,0.466667,0.550000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25619,440056,441974,Scarsdale Place Kensington Kensington and Chel...,1831,2016-04-05,8.1,Copthorne Tara Hotel London Kensington,United Kingdom,Queuing for breakfast and tea served from a j...,30,...,51.499046,-0.191707,UK,Positive,The location was very good Room was very clea...,"[location, good, room, clean, queuing, breakfa...","[copthorne, tara, hotel, london, kensington]",location good room clean queuing breakfast tea...,0.693333,0.845000
25620,200276,200658,5 Olympic Way Wembley Brent London HA9 0NP UK,832,2016-09-04,8.9,Novotel London Wembley,Ireland,No Negative,0,...,51.561127,-0.278726,UK,Positive,Staff were wonderful great location and beaut...,"[staff, wonderful, great, location, beautiful,...","[novotel, london, wembley]",staff wonderful great location beautiful break...,0.700000,0.790000
25621,53096,53096,146 Praed Street Westminster Borough London W2...,687,2016-01-12,8.2,Hilton London Paddington,United Kingdom,No Negative,0,...,51.515632,-0.175737,UK,Positive,Excellent Reception Staff -No Negative,"[excellent, reception, staff, negative]","[hilton, london, paddington]",excellent reception staff negative,0.575000,0.700000
25622,256317,256699,86 Rue Saint Denis 1st arr 75001 Paris France,18,2017-06-29,9.0,Snob Hotel by Elegancia,Spain,nowhere to store your toiletries in bathroom ...,16,...,48.862272,2.349571,France,Positive,good size for the price and it being paris sm...,"[good, size, price, paris, small, enough, spac...","[snob, hotel, elegancia]",good size price paris small enough space bathr...,0.280000,0.583333


#TF-IDF

In [30]:
#imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

In [27]:
tfidf = TfidfVectorizer(stop_words='english',smooth_idf=True)

# numbers to calculate similarities
tfidf_matrix = tfidf.fit_transform(hotel_sub_df['clean_keywords']).todense()

#calculate cosine matrix
cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(cosine_similarities)

indices = pd.Series(hotel_sub_df['Hotel_Name'])

C:\Users\neil.ralston.MAC\Anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


[[1.         0.00905824 0.         ... 0.0250256  0.         0.05616728]
 [0.00905824 1.         0.         ... 0.03792405 0.         0.04091212]
 [0.         0.         1.         ... 0.         0.         0.01802625]
 ...
 [0.0250256  0.03792405 0.         ... 1.         0.         0.05808119]
 [0.         0.         0.         ... 0.         1.         0.05256753]
 [0.05616728 0.04091212 0.01802625 ... 0.05808119 0.05256753 1.        ]]


In [31]:
#get recommendations based on tf-idf and cosine_similarities

def recommend_tf(Hotel_Name, cosine_similarities = cosine_similarities):
    
    recommended_hotel = []
    recommended_hotel_address = []
    recommended_hotel_rating = []
     
    idx = indices[indices == Hotel_Name].index[0]
    
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)
    
    top_10_indices = list(score_series.iloc[1:11].index)
    
    for i in top_10_indices:
        recommended_hotel.append(list(hotel_sub_df['Hotel_Name'])[i])
        recommended_hotel_address.append(list(hotel_sub_df['Hotel_Address'])[i])
        recommended_hotel_rating.append(list(hotel_sub_df['Average_Score'])[i])

    data = {'Hotel': recommended_hotel,
           'Address': recommended_hotel_address,
            'Rating': recommended_hotel_rating,
           }
    
    rec_df = pd.DataFrame(data)
    
    return rec_df

In [35]:
red_df.head()

NameError: name 'red_df' is not defined

In [34]:
recommend_tf('Hotel_Arena')

IndexError: index 0 is out of bounds for axis 0 with size 0